<a href="https://colab.research.google.com/github/yuto-kobayashi-1/signate-AIQuest/blob/main/20210922_AIquest_StockPredict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import lightgbm as lgb
from copy import deepcopy
import numpy as np
from datetime import datetime , date 
from sklearn.inspection import permutation_importance
from sklearn.model_selection import TimeSeriesSplit
from itertools import product

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
col_name = {'日付':'date',
          '店舗ID':'store_id',
          '商品ID':'goods_id',
          '商品価格':'price',
          '売上個数':'amount',
          '商品カテゴリID':'category_id'
          }

In [4]:
sales_data = pd.read_csv("/content/drive/MyDrive/signate/competiton/210919_AIquest/sales_history.csv",encoding="utf_8")
test_data = pd.read_csv("/content/drive/MyDrive/signate/competiton/210919_AIquest/test.csv",encoding="utf_8")

In [5]:
def preproces(df,col_name):
  df = df.rename(columns=col_name)
 
  # イレギュラーデータを削除する
  df = df[df['price']!=0]
  df = df[df['amount']>0]

  df['date'] = pd.to_datetime(df['date'])
  df['base_date'] = df['date'].dt.strftime("%Y%m")

  df['sales_amount'] = (df['price'] * df['amount']).astype('int')
 
  #月次単位にデータを集計
  df = df.drop(['price'],axis=1).groupby(['base_date','store_id','goods_id']).sum(['amount','sales_amount']).reset_index()
  df['av_price'] = (df['sales_amount'] /  df['amount'] )

  # df = df.groupby(['base_date','store_id','goods_id']).agg({'amount':'sum','price':'mean'}).reset_index()
  # df = df.rename(columns={'price':'av_price'})

   # 「日付」カラムの文字列から、「年」「月」の情報を抽出する
  df['year'] = df['base_date'].apply(lambda x: x[:4]) 
  df['month'] = df['base_date'].apply(lambda x: x[4:])
  
  #月ブロックの作成
  gp_time = df.groupby(['year', 'month']).count().reset_index()[['year', 'month']]
  # カラム名「月ブロック」として、通し番号をつける
  gp_time['base_date_count'] = list(range(len(gp_time)))

  # testデータ用に、評価対象期間である2019年12月(月ブロック: 23)のレコードを追加する
  gp_time = gp_time.append({
    'base_date_count': 23,
    'year': '2019',
    'month': '12'
  }, ignore_index=True)

  # 「月ブロック」カラムをsalesに統合する
  df = pd.merge(df, gp_time, on=['year', 'month'], how='left')

  all_combination = zero_padding_amount(df)

  all_combination = pd.merge(all_combination, gp_time, on=['base_date_count'], how='left')
  df = pd.merge(all_combination, df, on=['base_date_count', 'goods_id', 'store_id','year','month'], how='left')

  #発売後経過月数を算出
  tmp = df.groupby(['goods_id','base_date_count']).count().reset_index()[['goods_id','base_date_count']]
  tmp['num_of_month'] = tmp.groupby(['goods_id'])['base_date_count'].rank()
  df = df.merge(tmp,on=['goods_id','base_date_count'],how='left')
  
  # fillna_cols = ['amount','sales_amount']
  fillna_cols = ['amount']
  for col in fillna_cols:
    df[col] = df[col].fillna(0)

  # カテゴリーデータの登録
  category = pd.read_csv('/content/drive/MyDrive/signate/competiton/210919_AIquest/item_categories.csv')
  category = category.rename(columns=col_name)

  df = pd.merge(df,category,on=['goods_id'],how='left')

  return df , gp_time

In [6]:
def make_features(df):

  #グッズID単位の月間平均販売数
  tmp = df.groupby(['base_date_count','goods_id']).sum().reset_index()[['base_date_count','goods_id','amount']]
  tmp['mean_goods_amount'] = tmp['amount'] / len(df['store_id'].unique())
  df = pd.merge(df,tmp[['base_date_count','goods_id','mean_goods_amount']],on=['base_date_count','goods_id'],how='left')
  df['div_mean_goods_amount'] = df['amount'] - df['mean_goods_amount']
  df['div_mean_goods_amount'] = df['div_mean_goods_amount'].replace([np.inf, -np.inf], 0)

  #グッズID単位の平均価格
  tmp = df[df['av_price'] > 0] 
  tmp = tmp.groupby(['base_date_count','goods_id']).mean().reset_index()[['base_date_count','goods_id','av_price']]
  tmp['mean_av_price'] = tmp['av_price']
  df = pd.merge(df,tmp[['base_date_count','goods_id','mean_av_price']],on=['base_date_count','goods_id'],how='left')
  # df['div_mean_av_price'] = df['amount'] - df['mean_goods_amount']
  # df['div_mean_av_price'] = df['div_mean_goods_amount'].replace([np.inf, -np.inf], 0)

  #グッズID単位の平均価格との差
  df['diff_price'] = df['mean_av_price'] - df['av_price']

  #カテゴリID単位の月間平均販売数
  tmp = df_sales_data.groupby(['base_date_count','category_id']).sum().reset_index()[['base_date_count','category_id','amount']]
  tmp['mean_category_amount'] = tmp['amount'] / len(df['store_id'].unique())
  df = pd.merge(df,tmp[['base_date_count','category_id','mean_category_amount']],on=['base_date_count','category_id'],how='left')


  del tmp
  
  #移動平均作成
  df = df.sort_values(['base_date_count','goods_id','store_id'])
  period = [3,6,12]
  for i in period:
    # 販売個数の移動平均算出
    tmp = df[['goods_id','store_id','amount']].groupby(['store_id','goods_id'],group_keys=False).rolling(window=i).mean()['amount'].reset_index()
    df[f'mv{i}m_amount'] = tmp[['level_2','amount']].set_index('level_2')['amount']
    #移動平均乖離率の算出
    df[f'div_mv{i}m_amount'] = (df[f'mv{i}m_amount'] - df['amount'])/df['amount']
    df[f'div_mv{i}m_amount'] = df[f'div_mv{i}m_amount'].replace([np.inf, -np.inf], 0)
  
  
  #lag特徴量作成
  for i in range(2,13,1):
    lag = df.copy()
    lag['base_date_count'] = lag['base_date_count'] + i
    lag = lag.rename(columns={'amount': f'bf{i}m_amount'})
    lag = lag.rename(columns={'av_price': f'bf{i}m_av_price'})
    lag = lag.rename(columns={'mean_goods_amount': f'bf{i}m_mean_goods_amount'})
    lag = lag.rename(columns={'div_mean_goods_amount': f'bf{i}m_div_mean_goods_amount'})    

    lag = lag.rename(columns={'mean_category_amount': f'bf{i}m_mean_category_amount'})

    lag = lag.rename(columns={'mean_av_price': f'bf{i}m_mean_av_price'})
    lag = lag.rename(columns={'diff_price': f'bf{i}m_diff_price'})
    
    col = []
    for peripd in [3,6,12]:
      lag = lag.rename(columns={f'mv{peripd}m_amount': f'bf{i}m_mv{peripd}m_amount'})
      lag = lag.rename(columns={f'div_mv{peripd}m_amount': f'bf{i}m_div_mv{peripd}m_amount'})
      col.append(f'bf{i}m_mv{peripd}m_amount')
      col.append(f'bf{i}m_div_mv{peripd}m_amount')

    # lag = lag[['base_date_count', 'store_id', 'goods_id',f'bf{i}m_amount']
    lag = (lag[['base_date_count', 'store_id', 'goods_id'
            ,f'bf{i}m_amount',f'bf{i}m_av_price',f'bf{i}m_mean_goods_amount'
            ,f'bf{i}m_mean_category_amount',f'bf{i}m_div_mean_goods_amount'
            ,f'bf{i}m_mean_av_price',f'bf{i}m_diff_price'
            ]+col]
          )
    df = pd.merge(df, lag, on=['base_date_count', 'store_id', 'goods_id'], how='left', left_index=True)

  df = df.reset_index(drop=True)

 

  return df

In [46]:
def make_test_data(df_test,df_train,col_name):
  df_test = df_test.rename(columns=col_name)
  #初期値設定
  df_test['year'] = 2019
  df_test['month'] = 12
  df_test['amount'] = 0
  df_test['base_date_count'] = 23

  category = pd.read_csv('/content/drive/MyDrive/signate/competiton/210919_AIquest/item_categories.csv')
  category = category.rename(columns=col_name)
  df_test = pd.merge(df_test,category,on=['goods_id'],how='left')
  
  #traindata移動平均乖離率を算出
  #移動平均作成
  # df_train = df_train.sort_values(['base_date_count','goods_id','store_id'])

  period = [3,6,12]
  mv_col = []
  for i in period:
  #販売個数の移動平均用カラム追加
    mv_col.append(f'mv{i}m_amount')
  #移動平均乖離率のカラム追加
    mv_col.append(f'div_mv{i}m_amount')
 

  #店舗商品単位で過去販売数データを作成
  df_train = df_train[['store_id','goods_id','amount','base_date_count','av_price','mean_goods_amount','mean_category_amount','div_mean_goods_amount','mean_av_price','diff_price'] + mv_col]
  # print(df_train.columns)

  for i in range(2,13,1):
    lag = df_train.copy()
    lag['base_date_count'] = lag['base_date_count'] + i
    lag = lag.rename(columns={'amount': f'bf{i}m_amount'})
    lag = lag.rename(columns={'av_price': f'bf{i}m_av_price'})
    lag = lag.rename(columns={'mean_goods_amount': f'bf{i}m_mean_goods_amount'})
    lag = lag.rename(columns={'mean_category_amount': f'bf{i}m_mean_category_amount'})
    lag = lag.rename(columns={'div_mean_goods_amount': f'bf{i}m_div_mean_goods_amount'})  
    lag = lag.rename(columns={'mean_av_price': f'bf{i}m_mean_av_price'})  
    lag = lag.rename(columns={'diff_price': f'bf{i}m_diff_price'})

    col = []
    for peripd in [3,6,12]:
      lag = lag.rename(columns={f'mv{peripd}m_amount': f'bf{i}m_mv{peripd}m_amount'})
      lag = lag.rename(columns={f'div_mv{peripd}m_amount': f'bf{i}m_div_mv{peripd}m_amount'})
      col.append(f'bf{i}m_mv{peripd}m_amount')
      col.append(f'bf{i}m_div_mv{peripd}m_amount')

    # lag = lag[['base_date_count', 'store_id', 'goods_id',f'bf{i}m_amount']]
    lag = (lag[['base_date_count', 'store_id', 'goods_id'
            ,f'bf{i}m_amount',f'bf{i}m_av_price',f'bf{i}m_mean_goods_amount'
            ,f'bf{i}m_mean_category_amount',f'bf{i}m_div_mean_goods_amount'
            ,f'bf{i}m_mean_av_price',f'bf{i}m_diff_price'
            ]+col]
    )
  
    df_test = pd.merge(df_test, lag, on=['base_date_count', 'store_id', 'goods_id'], how='left', left_index=True)
  
  #発売経過月数追加
  tmp = df_test.groupby(['goods_id','base_date_count']).count().reset_index()[['goods_id','base_date_count']]
  tmp2 = df_train.groupby(['goods_id','base_date_count']).count().reset_index()[['goods_id','base_date_count']]
  tmp_m = pd.concat([tmp,tmp2])
  tmp_m['num_of_month'] = tmp_m.groupby(['goods_id'])['base_date_count'].rank()

  df_test = pd.merge(df_test,tmp_m,on=['base_date_count','goods_id'],how='left')
  del tmp,tmp2,tmp_m
  
  df_test = df_test.reset_index(drop=True)
  # #testデータに特徴量付与
  # df_test = df_test.merge(df_merge[['base_date_count', 'store_id', 'goods_id']+cols],how='left')

  return df_test

In [8]:
# def train(df,label_cols,features):
def trainCV(df,features):

  df = df.rename(columns = {'amount': 'y'})

  X =  df[features]
  y = df['y']

  for feature in features:
    X[feature] = X[feature].astype('float')

  # kf = KFold(n_splits=5,shuffle=True,random_state=0)
  kf = TimeSeriesSplit(n_splits=5)

  models = []
  for train_index, test_index in kf.split(X, y):
    X_train = X.iloc[train_index]
    y_train =  y.iloc[train_index]
    X_valid = X.iloc[test_index]
    y_valid = y.iloc[test_index]

    # X_train, y_train, X_valid, y_valid =  target_encording(X_train, y_train, X_valid, y_valid,label_cols)

    params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    # 'num_leaves': 64,
    # 'min_data_in_leaf': 20,
    # 'max_depth': 7,
    # 'verbose': 0,
  }

    train_data = lgb.Dataset(
      data=X_train, 
      label=y_train,
  )

    validation_data = lgb.Dataset(
      data=X_valid, 
      label=y_valid, 
  )

    model = lgb.train(
      params=params, 
      train_set=train_data, 
      num_boost_round=1000, 
      early_stopping_rounds=100,
      valid_sets=[validation_data],
  )

    models.append(model)

  return models 

In [9]:
# def train(df,label_cols,features):
def trainTS(df,features):

  df = df.rename(columns = {'amount': 'y'})

  idx_train={}
  idx_valid={}
  idx_train[0] = (df['base_date_count']<=9)
  idx_valid[0] = (df['base_date_count']==11)
  idx_train[1] = (df['base_date_count']<=19)
  idx_valid[1] = (df['base_date_count']==21)
  idx_train[2] = (df['base_date_count']>=12) & (df['base_date_count']<=19)
  idx_valid[2] = (df['base_date_count']==21)


  X =  df[features]
  y = df['y']

  for feature in features:
    X[feature] = X[feature].astype('float')

  # kf = KFold(n_splits=5,shuffle=True,random_state=0)
  # kf = TimeSeriesSplit(n_splits=5)

  models = []
  

  for i in range(len(idx_train)):
    print(i)
    X_train = X[idx_train[i]]
    y_train =  y[idx_train[i]]
    X_valid = X[idx_valid[i]]
    y_valid = y[idx_valid[i]]

    # X_train, y_train, X_valid, y_valid =  target_encording(X_train, y_train, X_valid, y_valid,label_cols)

    params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    # 'num_leaves': 64,
    # 'min_data_in_leaf': 20,
    # 'max_depth': 7,
    # 'verbose': 0,
  }

    train_data = lgb.Dataset(
      data=X_train, 
      label=y_train,
  )

    validation_data = lgb.Dataset(
      data=X_valid, 
      label=y_valid, 
  )

    model = lgb.train(
      params=params, 
      train_set=train_data, 
      num_boost_round=1000, 
      early_stopping_rounds=100,
      valid_sets=[validation_data],
  )

    models.append(model)

  return models 

In [10]:
def feature_importance(models,data,cols):
  importance = pd.DataFrame(columns=data[cols].columns)

  for model in models:
    imp_ = pd.Series(model.feature_importance(importance_type='gain'),index=data[cols].columns)
    importance = importance.append(imp_,ignore_index=True)

  importance = importance.mean()

  importance = pd.DataFrame(importance,columns={"importance"}).sort_values("importance",ascending=False)

  importance = pd.DataFrame(importance/importance.sum())
  return importance

In [11]:
def zero_padding_amount(df):
  all_combination = []

  # '月'、'商品ID'、 '店舗ID'の全組み合わせを作成する
  columns = ['base_date_count', 'goods_id', 'store_id']

  #product関数は指定された全組み合わせを作成する関数
  for i in range(22):
      train_block = df[df['base_date_count']==i]
      all_combination.append(np.array(list(product([i], train_block['goods_id'].unique(), train_block['store_id'].unique()))))
    
  all_combination = pd.DataFrame(np.vstack(all_combination), columns=columns)
  all_combination.sort_values(columns, inplace=True)

  return all_combination

前処理

In [12]:
df_sales_data , gp_time = preproces(sales_data,col_name)

In [44]:
df_sales_data.head()

,base_date_count,goods_id,store_id,year,month,base_date,amount,sales_amount,av_price,num_of_month,category_id
0,0,1000001,0,2018,01,201801,6.0,2520.0,420.0,1.0,100
1,0,1000001,1,2018,01,201801,2.0,650.0,325.0,1.0,100
2,0,1000001,2,2018,01,201801,1.0,420.0,420.0,1.0,100
3,0,1000001,3,2018,01,201801,2.0,840.0,420.0,1.0,100
4,0,1000001,4,2018,01,NaN,0.0,NaN,NaN,1.0,100


In [15]:
df_train = make_features(df_sales_data)

In [16]:
df_train[(df_train['goods_id'].isin([1000001]))&(df_train['store_id'].isin([0]))][['base_date_count','goods_id','store_id','av_price','mean_av_price','bf2m_diff_price']]

,base_date_count,goods_id,store_id,av_price,mean_av_price,bf2m_diff_price
0,0,1000001,0,420.000000,403.571429,NaN
83124,1,1000001,0,356.666667,388.333333,NaN
167076,2,1000001,0,420.000000,420.000000,-16.428571
252144,3,1000001,0,420.000000,406.428571,31.666667
335664,4,1000001,0,NaN,398.409091,0.000000
418878,5,1000001,0,420.000000,401.777778,-13.571429
503424,6,1000001,0,420.000000,403.235294,NaN
586656,7,1000001,0,420.000000,406.428571,-18.222222
667116,8,1000001,0,420.000000,420.000000,-16.764706
745362,9,1000001,0,NaN,417.948718,-13.571429


In [17]:
# df_train[(df_train['goods_id'].isin([1000001]))&(df_train['store_id'].isin([0]))][['base_date_count','goods_id','store_id','amount','mv3m_amount','div_mv3m_amount','bf2m_mv3m_amount','bf2m_div_mv3m_amount','div_mean_goods_amount','bf2m_div_mean_goods_amount','amount']]

In [47]:
df_test = make_test_data(test_data,df_train,col_name)

In [48]:
df_test

,index,goods_id,store_id,year,month,amount,base_date_count,category_id,bf2m_amount,bf2m_av_price,bf2m_mean_goods_amount,bf2m_mean_category_amount,bf2m_div_mean_goods_amount,bf2m_mean_av_price,bf2m_diff_price,bf2m_mv3m_amount,bf2m_div_mv3m_amount,bf2m_mv6m_amount,bf2m_div_mv6m_amount,bf2m_mv12m_amount,bf2m_div_mv12m_amount,bf3m_amount,bf3m_av_price,bf3m_mean_goods_amount,bf3m_mean_category_amount,bf3m_div_mean_goods_amount,bf3m_mean_av_price,bf3m_diff_price,bf3m_mv3m_amount,bf3m_div_mv3m_amount,bf3m_mv6m_amount,bf3m_div_mv6m_amount,bf3m_mv12m_amount,bf3m_div_mv12m_amount,bf4m_amount,bf4m_av_price,bf4m_mean_goods_amount,bf4m_mean_category_amount,bf4m_div_mean_goods_amount,bf4m_mean_av_price,...,bf10m_amount,bf10m_av_price,bf10m_mean_goods_amount,bf10m_mean_category_amount,bf10m_div_mean_goods_amount,bf10m_mean_av_price,bf10m_diff_price,bf10m_mv3m_amount,bf10m_div_mv3m_amount,bf10m_mv6m_amount,bf10m_div_mv6m_amount,bf10m_mv12m_amount,bf10m_div_mv12m_amount,bf11m_amount,bf11m_av_price,bf11m_mean_goods_amount,bf11m_mean_category_amount,bf11m_div_mean_goods_amount,bf11m_mean_av_price,bf11m_diff_price,bf11m_mv3m_amount,bf11m_div_mv3m_amount,bf11m_mv6m_amount,bf11m_div_mv6m_amount,bf11m_mv12m_amount,bf11m_div_mv12m_amount,bf12m_amount,bf12m_av_price,bf12m_mean_goods_amount,bf12m_mean_category_amount,bf12m_div_mean_goods_amount,bf12m_mean_av_price,bf12m_diff_price,bf12m_mv3m_amount,bf12m_div_mv3m_amount,bf12m_mv6m_amount,bf12m_div_mv6m_amount,bf12m_mv12m_amount,bf12m_div_mv12m_amount,num_of_month
0,0,1000001,0,2019,12,0,23,100,0.0,NaN,1.555556,406.833333,-1.555556,237.000000,NaN,0.333333,0.000000,0.500000,0.000000,0.666667,0.000000,1.0,250.0,0.833333,375.444444,0.166667,250.0,0.0,1.000000,0.000000,0.500000,-0.500000,0.666667,-0.333333,0.0,NaN,1.944444,513.388889,-1.944444,237.0,...,1.0,420.0,1.444444,648.722222,-0.444444,415.681818,-4.318182,1.000000,0.000000,1.000000,0.000000,1.500000,0.500000,2.0,420.0,1.277778,726.444444,0.722222,420.0,0.0,1.333333,-0.333333,1.166667,-0.416667,1.666667,-0.166667,0.0,NaN,2.111111,931.944444,-2.111111,420.0,NaN,0.666667,0.000000,1.333333,0.000000,2.000000,0.000000,23.0
1,1,1000001,1,2019,12,0,23,100,0.0,NaN,1.555556,406.833333,-1.555556,237.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.0,NaN,0.833333,375.444444,-0.833333,250.0,NaN,0.000000,NaN,0.000000,NaN,0.083333,0.000000,0.0,NaN,1.944444,513.388889,-1.944444,237.0,...,0.0,NaN,1.444444,648.722222,-1.444444,415.681818,NaN,0.000000,NaN,0.166667,0.000000,0.500000,0.000000,0.0,NaN,1.277778,726.444444,-1.277778,420.0,NaN,0.000000,NaN,0.166667,0.000000,0.583333,0.000000,0.0,NaN,2.111111,931.944444,-2.111111,420.0,NaN,0.333333,0.000000,0.333333,0.000000,0.750000,0.000000,23.0
2,2,1000001,2,2019,12,0,23,100,2.0,250.0,1.555556,406.833333,0.444444,237.000000,-13.000000,1.666667,-0.166667,1.166667,-0.416667,1.000000,-0.500000,0.0,NaN,0.833333,375.444444,-0.833333,250.0,NaN,1.000000,0.000000,1.000000,0.000000,0.916667,0.000000,3.0,250.0,1.944444,513.388889,1.055556,237.0,...,2.0,420.0,1.444444,648.722222,0.555556,415.681818,-4.318182,0.666667,-0.666667,1.000000,-0.500000,1.416667,-0.291667,0.0,NaN,1.277778,726.444444,-1.277778,420.0,NaN,0.333333,0.000000,1.333333,0.000000,1.250000,0.000000,0.0,NaN,2.111111,931.944444,-2.111111,420.0,NaN,0.666667,0.000000,1.833333,0.000000,1.333333,0.000000,23.0
3,3,1000001,3,2019,12,0,23,100,2.0,250.0,1.555556,406.833333,0.444444,237.000000,-13.000000,3.666667,0.833333,4.166667,1.083333,4.416667,1.208333,3.0,250.0,0.833333,375.444444,2.166667,250.0,0.0,4.666667,0.555556,4.166667,0.388889,4.833333,0.611111,6.0,250.0,1.944444,513.388889,4.055556,237.0,...,2.0,420.0,1.444444,648.722222,0.555556,415.681818,-4.318182,5.333333,1.666667,6.500000,2.250000,5.250000,1.625000,5.0,420.0,1.277778,726.444444,3.722222,420.0,0.0,7.000000,0.400000,7.166667,0.433333,5.083333,0.016667,9.0,420.0,2.111111,931.944444,6.888889,420.0,0.0,7.666667,-0.148148,7.000000,-0.222222,4.833333,-0.462963,23.0
4,4,1000001,4,2019,12,0,23,100,12.0,250.0,1.555

In [49]:
df_test['bf2m_mean_av_price']

0       237.000000
1       237.000000
2       237.000000
3       237.000000
4       237.000000
           ...    
3055    406.666667
3056    406.666667
3057    406.666667
3058    406.666667
3059    406.666667
Name: bf2m_mean_av_price, Length: 3060, dtype: float64

In [50]:
print(len(df_sales_data))
print(len(df_train))
print(len(test_data))
print(len(df_test))

1647990
1647990
3060
3060


In [51]:
#特徴量編集結果確認
i = 5
print(df_train.groupby('base_date_count').agg({'bf2m_amount': 'count','bf2m_av_price': 'count',f'bf{i}m_amount': 'count',f'bf{i}m_av_price': 'count'}))
print(df_test.groupby('base_date_count').agg({'bf2m_amount': 'count','bf2m_av_price': 'count',f'bf{i}m_amount': 'count',f'bf{i}m_av_price': 'count'}))
# print(df_train.groupby('base_date_count').agg({'bf2m_amount': 'count',f'bf{i}m_amount': 'count'}))
# print(df_test.groupby('base_date_count').agg({'bf2m_amount': 'count',f'bf{i}m_amount': 'count'}))

                 bf2m_amount  bf2m_av_price  bf5m_amount  bf5m_av_price
base_date_count                                                        
0                          0              0            0              0
1                          0              0            0              0
2                      70920          25923            0              0
3                      69660          24215            0              0
4                      70686          25951            0              0
5                      70542          23198        64674          24422
6                      69660          22149        64368          22941
7                      67158          22678        61866          23816
8                      66006          23884        60372          21190
9                      64296          23742        58428          19886
10                     63864          21092        59904          21004
11                     65682          21130        60552        

モデル構築

In [53]:
lag = []
#2～6か月前と12か月前の特徴量を利用する
month = [x for x in range(2,7,1)]
month.append(12)
for i in month:
  lag.append(f'bf{i}m_amount')
  lag.append(f'bf{i}m_av_price')
  lag.append(f'bf{i}m_mean_goods_amount')
  # lag.append(f'bf{i}m_mean_av_price')  
  # lag.append(f'bf{i}m_diff_price')
  # lag.append(f'bf{i}m_mean_category_amount')
  # lag.append(f'bf{i}m_div_mean_goods_amount')

  for peripd in [3,6,12]:
    lag.append(f'bf{i}m_mv{peripd}m_amount')
    # lag.append(f'bf{i}m_div_mv{peripd}m_amount')

features = ['store_id','goods_id','category_id','year','month','num_of_month'] + lag

In [54]:
features

['store_id',
 'goods_id',
 'category_id',
 'year',
 'month',
 'num_of_month',
 'bf2m_amount',
 'bf2m_av_price',
 'bf2m_mean_goods_amount',
 'bf2m_mv3m_amount',
 'bf2m_mv6m_amount',
 'bf2m_mv12m_amount',
 'bf3m_amount',
 'bf3m_av_price',
 'bf3m_mean_goods_amount',
 'bf3m_mv3m_amount',
 'bf3m_mv6m_amount',
 'bf3m_mv12m_amount',
 'bf4m_amount',
 'bf4m_av_price',
 'bf4m_mean_goods_amount',
 'bf4m_mv3m_amount',
 'bf4m_mv6m_amount',
 'bf4m_mv12m_amount',
 'bf5m_amount',
 'bf5m_av_price',
 'bf5m_mean_goods_amount',
 'bf5m_mv3m_amount',
 'bf5m_mv6m_amount',
 'bf5m_mv12m_amount',
 'bf6m_amount',
 'bf6m_av_price',
 'bf6m_mean_goods_amount',
 'bf6m_mv3m_amount',
 'bf6m_mv6m_amount',
 'bf6m_mv12m_amount',
 'bf12m_amount',
 'bf12m_av_price',
 'bf12m_mean_goods_amount',
 'bf12m_mv3m_amount',
 'bf12m_mv6m_amount',
 'bf12m_mv12m_amount']

In [55]:
models = trainTS(df_train,features)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0
[1]	valid_0's rmse: 3.86154
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's rmse: 3.80685
[3]	valid_0's rmse: 3.76438
[4]	valid_0's rmse: 3.71925
[5]	valid_0's rmse: 3.69028
[6]	valid_0's rmse: 3.66112
[7]	valid_0's rmse: 3.63684
[8]	valid_0's rmse: 3.61551
[9]	valid_0's rmse: 3.59758
[10]	valid_0's rmse: 3.58056
[11]	valid_0's rmse: 3.56762
[12]	valid_0's rmse: 3.55521
[13]	valid_0's rmse: 3.54568
[14]	valid_0's rmse: 3.53103
[15]	valid_0's rmse: 3.5238
[16]	valid_0's rmse: 3.51541
[17]	valid_0's rmse: 3.50622
[18]	valid_0's rmse: 3.49853
[19]	valid_0's rmse: 3.49403
[20]	valid_0's rmse: 3.48775
[21]	valid_0's rmse: 3.4842
[22]	valid_0's rmse: 3.47907
[23]	valid_0's rmse: 3.47648
[24]	valid_0's rmse: 3.47079
[25]	valid_0's rmse: 3.47045
[26]	valid_0's rmse: 3.4668
[27]	valid_0's rmse: 3.46867
[28]	valid_0's rmse: 3.46492
[29]	valid_0's rmse: 3.46507
[30]	valid_0's rmse: 3.46238
[31]	valid_0's rmse: 3.46042
[32]	valid_0's rmse: 3.45631
[33]	valid_0's rmse

In [56]:
importance = feature_importance(models,df_train,features)  

In [57]:
importance[importance['importance']>0].head(30)

,importance
bf2m_amount,0.251079
goods_id,0.178727
bf2m_mean_goods_amount,0.166684
month,0.082245
num_of_month,0.060575
store_id,0.048487
bf2m_av_price,0.029232
bf3m_mv3m_amount,0.020991
category_id,0.019226
bf2m_mv3m_amount,0.017976


In [58]:
rmses=[]
for model in models:
  rmses.append(model.best_score["valid_0"]["rmse"])
np.mean(rmses)

3.0749603962389664

予測

In [59]:
def predict(models,df,features):
  X = df[features]
  preds = np.zeros((len(df),len(models)))
  for n , model in enumerate(models):
    # pred = model.predict(df_test[cols],num_iteration=model.best_iteration)
    pred = model.predict(X,num_iteration=model.best_iteration)
    preds[:,n] = pred

  score = df.copy()
  score["pred"] = np.mean(preds,axis=1)
  
  return score

In [60]:
score = predict(models,df_test,features)

In [61]:
print(score['pred'].min())
print(score['pred'].max())

0.025330493399203638
16.475508146433526


In [62]:
score = score[['index','pred']]

In [63]:
score.to_csv("/content/drive/MyDrive/signate/competiton/210919_AIquest/submit.csv",header=False,index=False)